# Natural Language Generation
## Shairoz Sohail (ssohai3) - University of Illinois - CS410

-----------

The generation of human-like natural language has been the holy grail of a large body of artificial intelligence research in the past several decades. The fabeled 'Turing Test' for the capabilities of an AI agent was singularly based around the succesful generation of natural language. While there have been many critics of the Turing Test and of natural language generation as an AI benchmark, it has nonetheless proven to be a very challenging and uniquely human task. Language is full of many intricies and subtle cues that become huge pitfalls for automated systems, and the highly dynamic nature of human language makes it hard to reduce to fixed logical structures.

Over the past several years there have been significant breakthroughs in natural language generation, with one of the largest ones centering around the recent 'deep learning' revolution. Deep learning is a term given to the methodology of building learning algorithms using highly intricate neural network models, often with many ('deep') layers and complex architectures. A specific type of neural network that provides a time dependency strucutre, a recurrent neural network, has proven to be a leading tool for natural language generation. Hidden Markov Models, a more mathematically grounded method that uses conditional transition probabilities between words, has been used for natural language generation for a long time. In this guide, we take an overview of the task of natural language generation, describe these two leading approaches, and provide practical examples and outputs.

## Conditional Probability

Assume you are planning to go outside over the weekend and play a game of baseball. You are one homerun away from setting your personal record, however you are concered about the weather affecting your chances of hititng a homerun. You notice everytime it's a windy day, you have only a 2% chance of hitting a homerun. Normally, 5% of the hits you make are homeruns. Looking at the weather forecasts over the month, it looks like there's normally a 20% chance of this weekend being windy. We can set up this situation with random variables as follows:

Y = Hitting a home run

X = Is it a windy day?

P(Y) = 0.05

P(X) = 0.15

P(Y|X) = 0.02

Notice that if your home run odds were unaffected by the weather, X and Y would be independent of one another, hence P(Y|X) would equal P(Y)P(X)=(0.05)(0.15)=0.75 != 0.02

Similar to the example above, words in a sentence are not conditionally independent. Intuitively, if you see the word 'sunny' in a sentence, it is much more likely to see the word 'day' then the word 'buffalo'. Mathematically, we may state this as the following:

P('day' | 'sunny') > P('buffalo' | 'sunny')

### Given a text document, we may actually compute the probabilities shown above simply counting word occurances. Let's give it a try in Python using the NLTK (natural language toolkit) and Numpy (numerical computing) packages

In [71]:
from nltk import word_tokenize
import numpy as np
from collections import Counter
import random

In [72]:
sample_text = "the quick brown fox jumped over the lazy dog. Then the quick brown fox was run over by the quicker SUV"

def get_conditional_probabilities(text, target_word, return_succesors=False):
    succesors = []
    text = word_tokenize(text) # Splitting the text into bag of words
    # Finding occurances of the target_word in our text
    word_occurances = [i for i,x in enumerate(text) if x == target_word]
    # Retrieving 
    succesors = [text[i+1] for i in word_occurances]
    if return_succesors==True:
        return(Counter(succesors))
    for succesor in set(succesors):
        print("P(" + str(succesor) + " | " + target_word + ") = " + str((1/len(succesors))*succesors.count(succesor)))

In [73]:
get_conditional_probabilities(text = sample_text, target_word = "the")

P(quick | the) = 0.5
P(lazy | the) = 0.25
P(quicker | the) = 0.25


## Markov Chains

Given a conditional probability, P(B|A), we may imagine it as 'transitioning' to a state B, given we're in state A. For example P(rain | cloudy) can be seen as the probability of transitioning into rainy weather, given that the weather is currently cloudy. We can go wild with this idea and have tons of states and the probabilities of transitioning between them. Such a network of states and transition probabilities is called a Markov Chain. 


If we let all the words in our document be states, using the fact that we can estimate the conditional transition probabilities as above, we can visualize documents as large markov chains. 

<image>
    
Since the transition probabilities are not known exactly (they are 'hidden'), we can estimate them using a large corpus of documents. Consider the following extract from the Wikipedia article on Mixed nuts. Normally, we'd build the Markov Chain from all the words in the text, but that would lead to a really long and messy output. We choose two common words ('from' and 'nuts') and use them to build our mock Markov Chain

In [19]:
sample_text2 = '''
In Japan, mixed nuts are the second most popular table nuts, behind sweet chestnuts;
in the United States, they are second only to peanuts. Mixed nuts have also gained in popularity in the Argentinian 
market, which imported some $1.9 million in 1997, nearly half from the U.S. During the year 2002, 
U.S. companies sold $783 million of mixed nuts incorporating four or more varieties, mostly in canned form, 
representing hundreds of millions of pounds.The individual nuts that make up mixed nuts are harvested from 
all over the world. As a Dallas Fed publication supporting free trade puts it,"In the average can of mixed nuts, 
you might find almonds from Italy, walnuts from China, Brazil nuts from Bolivia, cashews from India, pistachios from 
Turkey, hazelnuts from Canada—a true international assortment."Label on a jar representing eight countries
This reality provides an incentive for nut salters to favor free trade for nuts, as opposed to nut farmers, 
who would generally support trade barriers. In fact, one historical argument for United States salters is 
that importing nuts can encourage domestic production, since mixed nuts provide a "wagon" on which 
everyone's sales ride. For example, cashews are not produced in North America, and it is necessary to 
import them because mixed nuts are essential to the sale of pecans, which are grown exclusively in North America
'''

In [20]:
 
    if vocabulary is None:
        vocabulary = word_tokenize(text)
    text = word_tokenize(text)
    for word_occurance in vocabulary:
        succesors = []
        # Finding occurances of the target_word in our text
        word_occurances = [i for i,x in enumerate(text) if x == word_occurance]
        # Retrieving 
        succesors = [text[i+1] for i in word_occurances]
        for succesor in succesors:
            print(word_occurance + '---' + str(round((1/len(succesors))*succesors.count(succesor),2)) + '-->' + str(succesor))

In [21]:
build_markov_chain(sample_text2, vocabulary = ['nuts', 'from'])

nuts---0.25-->are
nuts---0.25-->,
nuts---0.08-->have
nuts---0.08-->incorporating
nuts---0.08-->that
nuts---0.25-->are
nuts---0.25-->,
nuts---0.08-->from
nuts---0.25-->,
nuts---0.08-->can
nuts---0.08-->provide
nuts---0.25-->are
from---0.12-->the
from---0.12-->all
from---0.12-->Italy
from---0.12-->China
from---0.12-->Bolivia
from---0.12-->India
from---0.12-->Turkey
from---0.12-->Canada—a


By now you can probably guess, traversing through a Markov Chain will generate a sequence of words that should look similar to some of the text used to generate the Markov Chain. If we feed our MC a large enough collection of text, we may even see novel ideas and passages emerge. What if we uses a sample of 10,000 song lyrics from the 55,000 song lyrics dataset to generate our Markov Chain? We can easily test this idea by getting an efficient implementation of a Markov Chain and feeding it the dataset. There are some text artifacts from data formatting, but overall it should do a pretty good job. 

In [129]:
 class markov_langauge_model:
    def __init__ (self, order=1, seed=101):
        self.order = order
        self.group_size = self.order + 1
        self.text = None
        self.graph = {}
        return

    def train(self, filename): #receive the text file
        self.text = (filename).read().split()
        #self.text = self.text + self.text[: self.order]
        for i in range(0, len(self.text) - self.group_size):
            key = tuple(self.text[i: i + self.order])
            value = self.text[i + self.order]
            if key in self.graph:
                self.graph[key].append(value)
            else :
                self.graph[key] = [value]
        return
    
    def inspect_graph(self):
        print(self.graph.keys())

    def generate (self,length):
        random.seed(self.seed)
        index = random.randint (0, len (self.text) - self.order)
        result = self.text[index : index + self.order]
        for i in range (length):
            state = tuple (result [len (result) - self.order : ] )
            next_word = random.choice (self.graph [state] )
            result.append (next_word)
        s=" ".join (result [self.order : ] )
        s = (filter(lambda x: x.isalpha(), word_tokenize(s)))
        return(s)

In [130]:
new_model = markov_langauge_model()
new_model.train(filename=open('/Users/User/Desktop/Text_Generation/song_lyrics.txt','r'))
new_model.generate(150)

AttributeError: 'markov_langauge_model' object has no attribute 'seed'

### Markov Chain Order - Modeling longer ideas in text

Wonderful! So far we've looked at conditional probabilities and used those to build a Markov Chain for our text. A natural question to ask at this point would be: why are we only looking at the prior word to choose the next word? Humans have a much longer memory span then that, we don't just form our sentences by thinking about the previous word we spoke. What if we used conditional probabilities for pairs, or triplets of words? this is the concept of order in a Markov Chain. The chain we build above is of order 1. While the math becomes more complicated, we can build Markov Chains of higher order to generate more natural sounding text. Long-term dependencies are automatically captured in a LSTM neural network, however let's try to max out the Markov Chain method as much as we can first

In [13]:
new_model = markov_langauge_model(5)
new_model.train(filename=open('/Users/User/Desktop/Text_Generation/song_lyrics.txt','r'))
new_model.generate(150)

'Hey there, Mr. Cold, Cold Heart, I know I could be But it\'s just one of those days " "2835","I just want to be with you everywhere Oh I, I want to be with you everywhere Oh I, I want to be with you Don\'t give up on me I\'ll meet you when my chores are through I don\'t know how long I\'ll be But I\'m not gonna let you down Darling wait and see \'Cause between now and then, till I see you again I\'ll be loving you, love me If you get there before I do Don\'t give up on me Don\'t give up on me I\'ll meet you when my chores are through I don\'t know how long it\'ll last But I never regret the past When I wake up tomorrow, I\'ll be looking around for you I got no reason to be down at the station'




Finding the optimal order (we used 5 here) for human-like text is mostly a trial-and-error process, but even with this randomly selected order we got pretty awesome results. I for one, welcome our new robot overlords!

## Neural Networks

Some of the state-of-the-art results in machine learning have been accomplished with models known as artificial neural networks (ANNs). The very core ideas of these models is easy to state:


We'd like to predict a variable Y, given a variable X. Just like the above examples, we'd like to know P(Y|X). One way to do this is to try to model the function that converts the input X into Y, that is try to model f such that f(X)=Y. Assume you are given the following dataset:

    X        Y
   ---      ---
    8        80
    5        50
    4        40
    6        60
    7        70
    7.5      75
    9.2      92
    12       95
    13       85
    9        90
    

Reading off the first few results, we might think it would be a good general rule to say Y = X*10, or f(x) = 10x. This method does pretty good, yielding Y values {80,50,40,60,70,75,92,120,130,90}. The errors from the actual predictions are {0,0,0,0,0,0,0,25,45,0}, for an average error of 25+45/10 = 7. Not bad.

Let's plot our results

![Image of Yaktocat](xy_plot.jpg)


But it looks like our predictions are shooting off into infinity as X gets larger, while the actual Y seems bounded above by 100. While we'd need more data to make solid conclusions, it is not unlikely that this is data for hours of sleep vs. test score or something similar that has an upper bound. Since we can only get a straight line with the multiplication idea we had above, we need something a little more advanced to model the diminishing marginal returns of X. 

In essence, we need to introduce a function that produces a curved line instead of a straight one, that is to say a non-linear function. In a neural network, this is done simply by picking a non-linear function (such as x^2 or tanh(x)) and applying it after multiplying by the weight. This might yield a curve as follows

![Image of Yaktocat](xy_plot2.jpeg)

Now we know we can use two basic ideas to generate some pretty good predictions:
  
  1) Multiplying X by a number (call it W) = W*X
  
  2) Taking the result and passing it through a non-linear function P(WX)
  
Turns out, that's most of all we need! A neural network is simply a large connection of objects called neurons, each of which simply does the two steps above to it's input. Here's an example of a neural network with only a single neuron

![single_neural_nn](neural_network_single.png)

However, this is not a very powerful network! We should add more neurons! Since this will cause multiple Y's to be output, we need another neuron to aggregate the results.

![multiple_neuron_nn](neural_network_dense.png)

These networks decide their weights based on some pre-defined cost function, which captures the difference between the predicted values of Y and the actual values of Y. Since cost functions that are chosen are usually differentiable, we can nudge the weights in the right direction so that each time we decide new weights they make the prediction closer to the actual values of Y. This is known as gradient descent.

"Deep Learning" is simply the idea of creating neural networks such as the one above, but with multiple layers. This makes it easier to model complex dependencies and 'build-up' complex concepts from simple ideas. However, what do we feed this network to do langauge generation? We could feed it the prior word we've seen to predict the next one, however this will suffer from the same problem that the Markov Chian of order 1 suffered from - limited memory. We need a neural network that can take in sequential input (such as a sentence) and remember long-term dependencies... 

enter the 'Recurrent Neural Network'

*image courtesy of leonardo araujo santos

![recurrent_neural_network](https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/image_folder_6/recurrent.jpg)

In an extremely simplified view, wiring the neural network back into itself allows it to keep track of some of the previous inputs it's been fed. This may immediately raise many questions. How far back in inputs can it keep track of? How does it integrate this knowledge into new predictions? Unfortunately, the answers to these questions are not very straightforward and depend on how the network is wired up. For now, we can simply imagine RNN's working very similar to regular ANN's but with the addition of this circular wiring to help remember dependencies.
However, even this very intricate RNN has trouble remembering really long-term dependencies. A human is required to perform extensive tuning that depends on the type of dataset we're using and also takes a long time. A more intricate version of the RNN, known as a Long-Short-Term-Memory Recurrent Neural Network (or LSTM-RNN for short) takes care of this problem and is able to much more easily 'learn' the proper dependency structure in the text. 

We use a large sample of Nietzsche's (a prolific germal philosopher) essays to train the chracter-level LSTM-RNN from the Python Keras package. Note, this model takes several hours of training before it starts to produce barely coherent text, however the large amount of parameters allows it to more precisely learn the long-term dependencies and style in the text. 

In [ ]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
            
        # Stop training when output text begins looking coherent
        print()

In [68]:
for diversity in [0.2]:
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        print()
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

----- Generating with seed: "has the same origin, and follow the scen"

has the same origin, and follow the scend and and the strong to the presented to himself the proved and the most strong and also the soul, the standard themselves and interpreted the philosopher and standard and stand the power and and and the more the proved as the presented and the origination of the sense is and hadming the soul and something the standard of the soul, the solitary of the surprised the subtle and about the presented a


### Conclusion
In review, we went over the ideas of conditional probability and Markov Chains to demonstrate how a document can be seen as a graph of states (words) and the probabilities of transitioning between them. Given these, we can simply traverse the graph to generate new documents. We also reviewed the ideas behind Recurrent Neural Networks, and showed how they can use 'memory' cells to model arbitrarily long dependencies in text and generate new text which capture larger themes and topics such as those found in philosophy essays. These methods are currentely considered state-of-the-art in natural language generation and are finding new applications everyday (espacially in the form of voice assistants or automated customer-service centers). The flexibility of deep learning models like LSTM-RNN and the exponentially growing amount of text data being created every day is sure to see even more advances and breakthroughs in the exciting area of Natural Language Generation, to the point where not only is the Turing Test passed but computer-generated natural langauge becomes a critical part of global communication.


---

### References

- Markov Model of Natural Language, CS at Princeton
http://www.cs.princeton.edu/courses/archive/spr05/cos126/assignments/markov.html

- Language Modeling, Stanford CS Department
https://web.stanford.edu/class/cs124/lec/languagemodeling.pdf

- Recurrent Neural Networks, Stanford CS Department
http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture10.pdf

- Understanding LSTM Networks, Christopher Olah
http://colah.github.io/posts/2015-08-Understanding-LSTMs/

- Poetry in Python: Using Markov Chains to Generate Texts, Omer Nevo
http://il.pycon.org/2016/static/sessions/omer-nevo.pdf

- Keras Neural Network Examples
https://github.com/fchollet/keras/tree/master/examples

- 55,000 song lyrics dataset, Kaggle+LyricsFreak
https://www.kaggle.com/mousehead/songlyrics

